In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

In [2]:
def get_soup(url):
    # Initialize Browser
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    # Visit URL
    browser.visit(url)

    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    soup = bs(html, 'html.parser')

    return browser, soup

# NASA Mars News
* Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
# set the url https
url = 'https://redplanetscience.com/'

browser, soup = get_soup(url)

news = soup.find('div', id='news')

list_dates = news.find_all('div', class_='list_date')

maxdate = max([dt.datetime.strptime(ldate.text.strip(), '%B %d, %Y') for ldate in list_dates])


articles = news.find_all('div', class_='list_text')

for article in articles:
    list_date = article.select_one(".list_date").text.strip()

    if dt.datetime.strptime(list_date, '%B %d, %Y') == maxdate:
        newsa = article
        title = newsa.select_one(".content_title").text.strip()
        news_p = newsa.select_one(".article_teaser_body").text.strip()

news_dict = {'title':title,
                'news_p':news_p}

browser.quit()

news_dict

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\annar\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


{'title': "Mars InSight Lander to Push on Top of the 'Mole'",
 'news_p': 'Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.'}

# JPL Mars Space Images - Featured Image

* Visit the url for the Featured Space Image site here.

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a     variable called featured_image_url.

* Make sure to find the image url to the full size .jpg image.

* Make sure to save a complete url string for this image.

In [4]:
space_url = 'https://spaceimages-mars.com/'
browser, soup = get_soup(space_url)

featured_image = soup.find('img', class_='headerimage fade-in')

featured_image_url = space_url + featured_image['src']

browser.quit()

featured_image_url

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\annar\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


'https://spaceimages-mars.com/image/featured/mars3.jpg'

# Mars Facts

* Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter,     Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [5]:
mars_url = 'https://galaxyfacts-mars.com/'
# using pandas read html
tables = pd.read_html(mars_url)
len(tables)

mars_fact_df = tables[0]

mars_fact_df.columns = ['Description' , 'Mars', 'Earth']
mars_fact_df.set_index('Description' , drop=True , inplace= True)

mars_fact_df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


# Mars Hemispheres

* Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere     name. Use a Python dictionary to store the data using the keys img_url and title.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for   each hemisphere.

In [6]:
# set url
hem_url = 'https://marshemispheres.com/'
browser, soup = get_soup(hem_url)
# get soup


items = soup.find_all('div', class_="item")

hemisphere_urls = []

for item in items:
    hmsphere = {}
    name = item.h3.text
#     link = item.a['href']

    # get full image
    try:
        browser.links.find_by_partial_text(name).click()
        print(browser.url)
        html2 = browser.html
        imgsoup = bs(html2, 'html.parser')
        imgsoup
        img = imgsoup.find('img', class_="wide-image")
        
        hmsphere['title'] = name[:-9]
        hmsphere['img_url'] = hem_url + img['src']
          
    except:
        print("Could not get Image Link")   
        
    hemisphere_urls.append(hmsphere)    
    browser.back()

browser.quit()

print(hemisphere_urls)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\annar\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html
[{'title': 'Cerberus Hemisphere', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
